# Securing an atoti session with Auth0 using OIDC

Securing a session comes in two parts:
1. Implementing authentication mechanism to secure access to the session
2. Restricting access of modules or data access by users based on the roles granted

atoti supports multiple [authentication mechanisms](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.html) to cater to the needs of our end users. This notebook demonstrates how we use [OpenID Connect (OIDC)](https://openid.net/connect/) through [Auth0](https://auth0.com/) to implement security over atoti web application.

We will explore the topic with [atoti Business Edition](https://www.atoti.io/discover-atoti-for-business/) using the [Top 50 Fast Food](https://www.kaggle.com/datasets/stetsondone/top50fastfood) dataset from Kaggle, combined with its parent company information sourced from the internet.  

Check out [atoti documentation](https://docs.atoti.io) to read more on [securing the session](https://docs.atoti.io/latest/how_tos/security/secure_a_session.html#Configuring-the-authentication-mechanism).  

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=oidc-auth0" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover.png" alt="Try atoti"></a></div>

In [1]:
import os

import atoti as tt

## 1. Authentication setup in atoti  

We demonstrate below how we configure the authentication parameter of [`atoti.Session`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.__init__) for OIDC through Auth0. 

In Auth0, build a custom action under __Actions > Library__ to create a trigger for the `Login / Post Login` as follows (update namespace according to your environment):

```
exports.onExecutePostLogin = async (event, api) => {
  const namespace = 'https://dev-5m2svhd0.us.auth0.com';
  if (event.authorization) {
    api.idToken.setCustomClaim(`${namespace}/roles`, event.authorization.roles);
    api.accessToken.setCustomClaim(`${namespace}/roles`, event.authorization.roles);
  }
};
```

Thereafter, refer to the following for the setting mapping against Auth0:
<img src="https://data.atoti.io/notebooks/security/img/auth0-setup.png" width="80%" />  

`name_claim` could be email or name, depending on what you would like to see reflected in the application:  

<img src="https://data.atoti.io/notebooks/security/img/displayed_id.png" width="80%" />  

In [2]:
authentication = tt.OidcConfig(
    provider_id="auth0",
    issuer_url="https://dev-5m2svhd0.us.auth0.com/",
    client_id=os.environ["AUTH0_CLIENT_ID"],
    client_secret=os.environ["AUTH0_CLIENT_SECRET"],
    name_claim="name",
    scopes=["email", "profile", "username", "roles"],
    roles_claims=[
        "https://dev-5m2svhd0:us:auth0:com/roles",
    ],
)

Client ids and secrets should be kept private. As suggested in the [documentation](https://docs.atoti.io/latest/how_tos/security/secure_a_session.html#Configuring-the-authentication-mechanism), connection details can be read from environment variables for improved security.  

### 1.1. Instantiating session with fixed port

It is important to fix the port for atoti web application as we will require it to configure the application URIs in Auth0.  

<img src="https://data.atoti.io/notebooks/security/img/auth0-application-url.png" width="70%" />  

In Auth0, remember to include the following callback URL with the [provider id](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html#atoti.OidcConfig.provider_id) set to `auth0`:

`f"{session_url}/login/oauth2/code/{provider_id}"`

In [3]:
session = tt.Session(
    port=10011, authentication=authentication, user_content_storage="./content"
)

## 2. Roles management in atoti

### 2.1. atoti reserved roles  

The below roles are reserved in atoti and should not be altered by users:
- ROLE_ADMIN: able to access all objects in the web application
- ROLE_USER: able to access all data by default. Access to objects such as dashboards, folders, widgets etc is only upon sharing access granted to role.
- ROLE_SHARE: able to share objects such as dashboards, folders, widgets and filters.  

While both _atoti_admin_ and _atoti_user_ are able to access all data, _atoti_admin_ is able to access all objects such as folders and dashboards.  
_atoti_user_ is able to access only the objects created by the user him/herself. Objects created by other users can only be access upon shared access granted.  

Both _atoti_admin_ and _atoti_user_ will not be able to share objects (via the "Share" icon as shown below) unless granted the role __ROLE_SHARE__.  

<img src="https://data.atoti.io/notebooks/security/img/sharing.png" width="70%"/>

### 2.2. Role creation with restrictions  

Data restriction is based on users' requirement. In our use case, we assumed two groups of users with data access limited to those of their parent company:
- users belonging to parent company _Inspire Brands_
- users belonging to parent company _Restaurant Brands International Inc._

Therefore, we will create two roles to apply the restrictions based on the values of the `parent_company` hierarchy.  

__NOTE:__  
- We can skip role creation if there are no restrictions imposed on the role. 
- The value provided under the restrictions is cap-sensitive.

In [4]:
role_inspire = session.security.create_role(
    "ATOTI_ROLE_INSPIRE", restrictions={"parent_company": ["Inspire Brands"]}
)

role_restaurant = session.security.create_role(
    "ATOTI_ROLE_RESTAURANT",
    restrictions={"parent_company": ["Restaurant Brands International Inc."]},
)

#### 2.2.1. Restricted access from combination of roles

Multiple roles can be assigned to the same user. To demonstrate how the access will change when this happens, we create some other roles that restrict data access by the restaurant category, i.e. hierarchy `category`.

In [5]:
role_team_burger = session.security.create_role(
    "ATOTI_ROLE_BURGER", restrictions={"category": ["burger"]}
)

role_team_sandwich = session.security.create_role(
    "ATOTI_ROLE_SANDWICH", restrictions={"category": ["sandwich"]}
)

role_team_snack = session.security.create_role(
    "ATOTI_ROLE_SNACK", restrictions={"category": ["snack"]}
)

When combined with the restricted role on the `parent_company`, user's access will be further restricted to based on the restriction of the added role.  

For instance, users who are assigned the role __ATOTI_ROLE_BURGER__ will be able to access all the data under _burger_ category restaurants, regardless of the parent companies.  

However, when the same user is also granted the role __ATOTI_ROLE_INSPIRE__, then the user can only access data of restaurants under parent company _Inspire Brands_ that is of category _burger_. 

### 2.3. Role assignment in atoti

#### 2.3.1. Assign default roles to authenticated users

We could default the roles such as __ROLE_USER__ and __ROLE_SHARE__ for all users who are logged in successfully.  

In [6]:
session.security.oidc.default_roles.update(["ROLE_USER", "ROLE_SHARE"])

__BE CAUTION__ when granting __ROLE_USER__ to users by default as users with this role will be able to access the application and the available data.  
It might be a better option to grant __ROLE_USER__ individually to the rightful users in the next step.

#### 2.3.2. Map roles between Authentication Provider and atoti

The names in the authentication provider can be different from those in atoti.    
Roles are assigned to users in the authentication provider and associated to the atoti roles in the `role_mapping` below.

In [26]:
session.security.oidc.role_mapping.update(
    {
        # atoti reserved roles
        "admin": ["ROLE_ADMIN"],
        "atoti_user": ["ROLE_USER"],
        "ROLE_SHARE": ["ROLE_SHARE"],
        # user-defined roles
        "ROLE_INSPIRE": [role_inspire.name],
        "ROLE_RESTAURANT": [role_restaurant.name],
        "ROLE_BURGER": [role_team_burger.name],
        "ROLE_SANDWICH": [role_team_sandwich.name],
        "ROLE_SNACK": [role_team_snack.name],
    }
)

While a role from the authentication provider can be mapped to multiple atoti roles, it is easier to modify user's access by updating the roles in the authentication provider. It is a design consideration whether to have a one-to-one role map or to grant multiple atoti roles to an authentication provider role.  

E.g. The mapping `AUTH_ROLE_USER: [ATOTI_ROLE_USER, ATOTI_ROLE_SHARE]` meant that all the users with __atoti user__ role assigned will be able to do sharing.  
Alternatively, it could be implemented in Auth0 as shown below, where individual users are granted the roles __atoti user__ and __ROLE_SHARE__.  

<img src="https://data.atoti.io/notebooks/security/img/user_roles.png" width="70%" />

## 3. Convergent with atoti Community Edition

We can now proceed with the usual data loading into atoti table, cube and measures creation.  

### 3.1. Data loading

In [8]:
base_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data//parent_co.csv",
    table_name="parent_co",
    keys=["company", "parent_company"],
    process_quotes=True,
)
base_tbl.head()

,,name
parent_company,company,
Inspire Brands,arbys,Arby's
Focus Brands,auntie_annes,Auntie Anne's
Inspire Brands,baskin_robbins,Baskin-Robbins
Jordan Company,bojangles,Bojangles
Durational Capital Management,bojangles,Bojangles


In [9]:
enrichment_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/top_50_fast_food_US.csv",
    table_name="top_50",
    keys=["company"],
)
enrichment_tbl.head()

,category,sales_in_millions_2019,sales_per_unit_thousands_2019,franchised_units_2019,company_owned_units_2019,total_units_2019,unit_change_from_2018
company,,,,,,,
mcdonalds,burger,40413,2912,13154,692,13846,-66
starbucks,snack,21550,1454,6768,8273,15041,216
chick_fil_a,chicken,11000,4517,2500,0,2500,130
taco_bell,global,11000,1502,6622,467,7089,181
burger_king,burger,10300,1399,7294,52,7346,16


In [10]:
base_tbl.join(enrichment_tbl)

### 3.2. Cube creation

In [11]:
cube = session.create_cube(base_tbl, "Fast food analysis")

### 3.3 Measures creation

In [12]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [13]:
m["sales_in_millions"] = tt.agg.sum(enrichment_tbl["sales_in_millions_2019"])
m["sales_per_unit_thousands"] = tt.agg.sum(
    enrichment_tbl["sales_per_unit_thousands_2019"]
)
m["franchised_units"] = tt.agg.sum(enrichment_tbl["franchised_units_2019"])
m["company_owned_units"] = tt.agg.sum(enrichment_tbl["company_owned_units_2019"])
m["total_units"] = tt.agg.sum(enrichment_tbl["total_units_2019"])
m["unit_change_from_2018"] = tt.agg.sum(enrichment_tbl["unit_change_from_2018"])

In [14]:
m["% franchised"] = m["franchised_units"] / m["total_units"]
m["% franchised"].formatter = "DOUBLE[0.00%]"

## 4. Test login and access management in web application

Try out any of these users:

___Administrator___
- atoti_admin

___Generic user___
- atoti_user

___Inspire Brands users___
- Inspire_user1 (Access for restaurants of category Burger only)
- Inspire_user2 (Access for restaurants of category Sandwich and Snack only)
- Inspire_manager

___Restaurant Brands International LLC users___
- Restaurant_user1 (Access for restaurants of category Burger only)
- Restaurant_user2 (Access for restaurants of category Sandwich and Snack only)
- Restaurant_manager

In [15]:
session.link()

Open the notebook in JupyterLab with the atoti extension enabled to see this link.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=oidc-auth0" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try atoti"></a></div>